This is my first attempt to solve the Spaceship Titanic Data science competition on the data science platform Kaggle.

# Preliminary: imports and load the data

In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os

In [7]:
# variable to determine whether the notebook is running on Collab or not
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

Please make sure to download the data and have it in the same directory as this notebook when running it.

In [8]:
import os
from zipfile import ZipFile

CURRENT_DIR = "sample_data" if IN_COLAB else os.getcwd() 
file_name = "spaceship-titanic.zip"
file_name_no_zip = "spaceship-titanic"
loc = os.path.join(CURRENT_DIR, file_name)
# opening the zip file in READ mode

if not os.path.isdir(os.path.join(CURRENT_DIR, file_name_no_zip)):
    with ZipFile(loc, 'r') as zip_ref:
        zip_ref.extractall()
        print('Done!')

Done!


In [9]:
train_loc = os.path.join(CURRENT_DIR, "train.csv")
test_loc = os.path.join(CURRENT_DIR, "test.csv")

df = pd.read_csv(train_loc)
df_test = pd.read_csv(test_loc)

# Cleaning the data
In this section, I proceed with having a general idea about the dataset with minimal data exploration in order to certain issues such as: 
* missing values 
* outliers
* multicollinearity
* 